In [ ]:
import pandas as pd

# URL do dataset Wine
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'

# Nome das colunas em português BR
column_names_pt_br = [
    'classe',
    'alcool',
    'acido_malico',
    'cinzas',
    'alcalinidade_de_cinzas',
    'magnesio',
    'fenois_totais',
    'flavanoides',
    'fenois_nao_flavanoides',
    'proantocianinas',
    'intensidade_de_cor',
    'matiz',
    'od280_od315_de_vinhos_diluidos',
    'prolina'
]

# Ler o arquivo CSV com as colunas especificadas, definindo a coluna 'classe' como object
vinhos = pd.read_csv(url, names=column_names_pt_br, dtype={'classe': object})


In [ ]:

X = vinhos.drop(columns=['classe'])
y = vinhos['classe']

# Verificando as dimensões de X e y
print("Dimensões de X:", X.shape)
print("Dimensões de y:", y.shape)


Dimensões de X: (178, 13)
Dimensões de y: (178,)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Função para realizar os experimentos
def realizar_experimentos(random_states, k_values):
    resultados = []

    for random_state in random_states:

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

        for k in k_values:
            # Treinando o modelo com KNN
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)


            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)


            resultados.append((random_state, k, accuracy))

    return resultados


random_states = [42, 17, 24]
k_values = [3, 5]


resultados = realizar_experimentos(random_states, k_values)


import pandas as pd
df_resultados = pd.DataFrame(resultados, columns=['Random State', 'k', 'Accuracy'])

# Exibindo os resultados
print("Resultados dos Experimentos:")
print(df_resultados)

# Identificando a melhor configuração
melhor_configuracao = df_resultados.loc[df_resultados['Accuracy'].idxmax()]
print("\nMelhor Configuração:")
print(melhor_configuracao)


Resultados dos Experimentos:
   Random State  k  Accuracy
0            42  3  0.740741
1            42  5  0.740741
2            17  3  0.648148
3            17  5  0.666667
4            24  3  0.740741
5            24  5  0.666667

Melhor Configuração:
Random State    42.000000
k                3.000000
Accuracy         0.740741
Name: 0, dtype: float64


In [ ]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd


def cross_validation_kfold(random_state, k_values):

    kf = KFold(n_splits=10, shuffle=True, random_state=random_state)

    resultados = []

    for k in k_values:
        acuracias = []

        for train_index, test_index in kf.split(X):

            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]


            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)

            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            acuracias.append(accuracy)

        media = np.mean(acuracias)
        desvio_padrao = np.std(acuracias)

        resultados.append((random_state, k, media, desvio_padrao))

    return resultados


random_states = [42, 17, 24]
k_values = [3, 5]


todos_resultados = []
for rs in random_states:
    resultados_rs = cross_validation_kfold(rs, k_values)
    todos_resultados.extend(resultados_rs)


df_resultados = pd.DataFrame(todos_resultados, columns=['Random State', 'k', 'Média Accuracy', 'Desvio Padrão'])

# Exibindo os resultados
print("Resultados de Cross-Validation com KFold:")
print(df_resultados)


Resultados de Cross-Validation com KFold:
   Random State  k  Média Accuracy  Desvio Padrão
0            42  3        0.703595       0.139187
1            42  5        0.664052       0.100075
2            17  3        0.698693       0.104030
3            17  5        0.687582       0.112118
4            24  3        0.712745       0.096816
5            24  5        0.712745       0.093973


In [ ]:

def analise_resultados(df_resultados):
    analise = []

    grouped = df_resultados.groupby('Random State')

    for random_state, group in grouped:

        for k in [3, 5]:
            dados_k = group[group['k'] == k]
            media_acuracia = dados_k['Média Accuracy'].values[0]
            desvio_padrao = dados_k['Desvio Padrão'].values[0]

            analise.append((random_state, k, media_acuracia, desvio_padrao))


        media_k3 = group[group['k'] == 3]['Média Accuracy'].values[0]
        media_k5 = group[group['k'] == 5]['Média Accuracy'].values[0]

        if media_k3 > media_k5:
            melhor_k = 3
        elif media_k3 < media_k5:
            melhor_k = 5
        else:
            melhor_k = "Empate"


        analise.append((random_state, "Melhor k", melhor_k, None))

    return pd.DataFrame(analise, columns=['Random State', 'k', 'Média Accuracy', 'Desvio Padrão'])

df_analise = analise_resultados(df_resultados)

print("Análise dos Resultados do K-Fold Cross-Validation:")
print(df_analise)


Análise dos Resultados do K-Fold Cross-Validation:
   Random State         k Média Accuracy  Desvio Padrão
0            17         3       0.698693       0.104030
1            17         5       0.687582       0.112118
2            17  Melhor k              3            NaN
3            24         3       0.712745       0.096816
4            24         5       0.712745       0.093973
5            24  Melhor k         Empate            NaN
6            42         3       0.703595       0.139187
7            42         5       0.664052       0.100075
8            42  Melhor k              3            NaN
